# Urban Observatory Newcastle University
- Author: Sedar Olmez
- Data source: http://newcastle.urbanobservatory.ac.uk/
- Description: Try downloading live sensory data from Newcastle to gain insight on the movement of people across the city. This will then be part of a pipeline system that inputs the data into an agent based model to simulate. 

In [3]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

ImportError: No module named 'pandas.plotting'

In [ ]:
import requests
import pandas as pd
import io

In [ ]:
def GeoJSON(data={}, metadata={}):
    bundle = {
            'application/geo+json': data,
            'text/plain': '<IPython.display.GeoJSON object>'
    }
    metadata = {
        'application/geo+json': metadata
    }
    display(bundle, metadata=metadata, raw=True)

# Retrieve sensor information:

In [ ]:
sensor_params = dict(
    sensor_type = 'Walking North'
)

r = requests.get('http://uoweb3.ncl.ac.uk/api/v1.1/sensors/csv/',sensor_params)

sensor_info = pd.read_csv(io.StringIO(r.text) )
sensor_info

Now that we have all sensors of type Walking North. We can try get the data these sensors capture. 

In [ ]:
# Time to plot the sensors on a map.
import smopy
import matplotlib.patheffects as pe

In [ ]:
black_box = (
    min(sensor_info['Sensor Centroid Latitude']),
    min(sensor_info['Sensor Centroid Longitude']),
    max(sensor_info['Sensor Centroid Latitude']),
    max(sensor_info['Sensor Centroid Longitude']),
)

map = smopy.Map(black_box, z = 17)
ax = map.show_mpl(figsize=(15, 15))

for i, row in sensor_info.iterrows():
    x, y = map.to_pixels(row['Sensor Centroid Latitude'], row['Sensor Centroid Longitude'])
    ax.plot(x, y, 'ob', ms = 20, mew = 2);

# Setting up data parameters

In [ ]:
import datetime

api_date_format = "%Y%m%d%H%M%S"

start_time = datetime.datetime(2019, 9, 18, 0)
end_time = datetime.datetime(2019, 9, 19)

data_parameters = dict(
    data_variable = 'Walking North',
    agg_method = 'median',
    agg_period = '5mins',
    starttime = start_time.strftime(api_date_format),
    endtime = end_time.strftime(api_date_format)
)

data_parameters.update(sensor_params)
data_parameters

**Get aggregated data**

In [ ]:
r = requests.get('http://uoweb3.ncl.ac.uk/api/v1.1/sensors/data/agg/csv/',data_parameters)
r

**Read the data into a dataframe**

In [ ]:
pedestrian_step_count_data = pd.read_csv(io.StringIO(r.text))
pedestrian_step_count_data.head(10)

**Plotting the data**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(30, 10))
for sensor_name, sensor_data in pedestrian_step_count_data.groupby('Sensor Name'):
    
    datetimes = pd.to_datetime(sensor_data['Timestamp'])
    plt.plot(datetimes, sensor_data['Value'], label = sensor_name)

    plt.legend()

**Plot of median**

In [ ]:
footfall_data_median = pedestrian_step_count_data.copy()
footfall_data_median.index = pd.to_datetime(footfall_data_median['Timestamp'])
footfall_data_median = footfall_data_median.resample('900s').median()

fig, ax = plt.subplots(figsize=(30,10))

plt.plot(footfall_data_median.index,footfall_data_median['Value'])

plt.legend()

In [ ]:
sensor_data.info()